In [1]:
library("Seurat", help, pos = 2, lib.loc = NULL)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t




In [2]:
setwd("../../DATA/H3k4me3/")

In [3]:
# library("Matrix", help, pos = 2, lib.loc = NULL)
# genescore = readMM("GeneScore.mtx")
# cells = read.csv("GeneScoreCells.csv")
# cells = cells$X0
# genes = read.csv("GeneScoreGenes.csv")
# genes = genes$index
# rownames(genescore) <- cells
# colnames(genescore) <- genes
# genescore = genescore[colnames(rna),]
# genescore <- CreateSeuratObject(counts = t(genescore))

In [4]:
# library(zellkonverter)
# rna <- readH5AD("rna-pp.h5ad")
# rna <- CreateSeuratObject(counts = rna@assays@data$X)

In [5]:
# saveRDS(genescore, file = "GeneScore.rds")
# saveRDS(rna, file = "rna.rds")

In [6]:
rna <- readRDS("rna.rds")
genescore <- readRDS("GeneScore.rds")

In [7]:
rna$omic <- "rna"
genescore$omic <- "atac"

In [8]:
rna <- NormalizeData(rna, verbose=FALSE)
rna <- FindVariableFeatures(rna, verbose=FALSE, nfeatures=3000)
rna <- ScaleData(rna, verbose=FALSE)

In [9]:
genescore <- NormalizeData(genescore)
genescore <- FindVariableFeatures(genescore, verbose=FALSE, nfeatures=3000)
genescore <- ScaleData(genescore)

Normalizing layer: counts



Centering and scaling data matrix



In [10]:
# Combined
Int.anchors <- FindIntegrationAnchors(object.list = list(rna, genescore))
combined <- IntegrateData(anchorset = Int.anchors)
DefaultAssay(combined) <- "integrated"
combined <- ScaleData(combined, verbose = FALSE)
combined <- RunPCA(combined, verbose = FALSE)
combined <- RunUMAP(combined, reduction = "pca", dims = 1:50)
combined <- FindNeighbors(combined, reduction = "pca", dims = 1:50)

Warning message in CheckDuplicateCellNames(object.list = object.list):
“Some cell names are duplicated across objects provided. Renaming to enforce unique cell names.”


Computing 2000 integration features

Scaling features for provided objects

Warning message:
“Different features in new layer data than already exists for scale.data”
Warning message:
“Different features in new layer data than already exists for scale.data”
Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 29418 anchors

Filtering anchors

	Retained 3924 anchors

Merging dataset 2 into 1

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Warning message:
“Layer counts isn't present in the assay object; returning NULL”
Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”
23:13:19 UMAP embedding parameters a = 0.9922 b = 1.112

23:1

In [15]:
combined <- FindClusters(combined)
print(length(unique((combined$seurat_clusters))))

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 19084
Number of edges: 726270

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8260
Number of communities: 19
Elapsed time: 2 seconds


2 singletons identified. 17 final clusters.



[1] 17


In [16]:
embed = combined@reductions$pca@cell.embeddings
umap <- combined@reductions$umap@cell.embeddings
cluster <- combined$seurat_clusters

In [13]:
# write.csv(file="rep1/Seurat_embed.csv", embed)
# write.csv(file="rep1/Seurat_umap.csv", umap)
# write.csv(file="rep1/Seurat_cluster.csv", cluster)